This is the file containing the code for the different formulations in our project, chronological order (from simpler to more complex).

In [1]:
using DataFrames, CSV, JuMP, Gurobi, Plots, Random

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

# Parameters and functions

In [3]:
# global variables

# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

In [4]:
function GetC()

    W_ = 19

    C = zeros((D*(W_+1), D*(W_+1)))

    for t in 1:(W_*D)
        for t_ in 1:(W_*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [5]:
function calendar_week(x,week, N)
    df = DataFrame(Player=1:N, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0, Monday=0.0)
    for i=1:N
        for j=1:N
            for d=1:D
                if x[i,j,week,d] == 1.0
                        df[i, 1+d] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [6]:
# compute the value of the objective function based on the calendar
function ComputeObjectiveValue(calendar, W, N)

    # define empty matrix to concatenate all the other weeks
    m = Array{Float64}(undef, N, 0)
    
    # build complete calendar
    for w in 1:W
        week = Matrix(calendar[w][:,2:end])
        m = [m week]
    end

    return ComputeObjectiveValueMatrix(m, W, N)
end

ComputeObjectiveValue (generic function with 1 method)

In [7]:
# compute the value of the objective function based on the calendar
function ComputeObjectiveValueMatrix(m, W, N)

    # define total cost for each team
    team_total_cost = []

    # for each team 
    for i = 1:N
        # find index of nonzer elements, they correspond to the days in which the team plays
        team_plays = findall(x -> x != 0, m[i, :])

        # we will find W non-zero elements, since each team plays against all the other teams
        # for i = 1:W, return GetC()[team_plays[i], team_plays[i+1]] and sum
        append!(team_total_cost, sum([GetC()[team_plays[i], team_plays[i+1]] for i = 1:W-1]))
    end

    # total cost of the calendar
    return sum(team_total_cost)
end

ComputeObjectiveValueMatrix (generic function with 1 method)

In [8]:
function GetCalendarMatrix(calendar, W, N)

    # define empty matrix to concatenate all the other weeks
    m = Array{Float64}(undef, N, 0)
    
    # build complete calendar
    for w in 1:W
        week = Matrix(calendar[w])[:, 2:end] # first column is the index
        m = [m week]
    end

    return m
end

GetCalendarMatrix (generic function with 1 method)

In [26]:
function random_restart(calendar_matrix, obj_calendar, Y_old, approach, time)
    
    # randomly pick a number of weeks to reoptimize (in addition to initial week)
    n = rand(1:4)

    # randomly choose initial week
    Win_ = rand(1:19-n)
    Wend_ = Win_ + n

    # get portion of the calendar matrix corresponding the the weeks to reoptimize
    calendar_reoptimize = calendar_matrix[:, 7 * (Win_-1) + 1 : 7 * Wend_]

    # for each team, get the list of opponents in these weeks
    for i in 1:20
        plays_against = [Int(element) for element in calendar_reoptimize[i, :] if element != 0]
        # modify Y_old accordingly, putting it to 0 so that the teams can be made play again
        for element in plays_against
            Y_old[i, element] = 0
        end
    end

    if approach == 3
        # call the model
        X_, Y_old, time = Optimize4WeeksAndFreeze(Y_old, Win_, Wend_, time)
    elseif approach == 4
        # call the model
        X_, Y_old, time = Optimize4WeeksAndFreezeFirst(Y_old, Win_, Wend_, true, time)
    end

    # get new calendar
    calendar_reoptimize = []

    for w in Win_:Wend_
        push!(calendar_reoptimize, calendar_week(X_, w, N))
    end

    calendar_reoptimize = GetCalendarMatrix(calendar_reoptimize, Wend_ - Win_ + 1, N)

    # update calendar
    calendar_new = calendar_matrix
    calendar_new[:, 7 * (Win_-1) + 1 : 7 * Wend_] = calendar_reoptimize

    # find objective value corresponding to new calendar
    obj_calendar_new = ComputeObjectiveValueMatrix(calendar_new, W, N)

    println("Objective value of new calendar: ", obj_calendar_new)

    # if new calendar is better, update the calendar
    if obj_calendar_new < obj_calendar
        calendar_matrix = calendar_new
        obj_calendar = obj_calendar_new
    end

    return calendar_matrix, obj_calendar, Y_old, time
end

random_restart (generic function with 1 method)

# Models

## Preliminary attempt 1: optimal formulation, show it scales with 6 teams

In [10]:
function optimal_formulation(N_teams, Win, Wend)

    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 60)

    # PARAMETERS
    C = GetC()
    N = N_teams # teams
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D))

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d_])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d_] - 1)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    # each team plays N - 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == N - 1)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    # OPTIMIZE
    solvetime = @elapsed optimize!(model)


    return value.(x), solvetime
end

optimal_formulation (generic function with 1 method)

In [13]:
N_teams = 6
Win_ = 1
Wend_ = N_teams - 1
X_p1, time_p1 = optimal_formulation(N_teams, Win_, Wend_);

In [12]:
# time it takes
time_p1

2.1775751

Just care that it's fast, don't care about solution because not comparable

In [76]:
N_teams_8 = 8
Win_ = 1
Wend_ = N_teams_8 - 1
X_p1_8, time_p1_8 = optimal_formulation(N_teams_8, Win_, Wend_)
time_p1_8

In [ ]:
N_teams_10 = 10
Win_ = 1
Wend_ = N_teams_10 - 1
X_p1_10, time_p1_10 = optimal_formulation(N_teams_10, Win_, Wend_)
time_p1_10

10.2022051

In [17]:
N_teams_12 = 12
Win_ = 1
Wend_ = N_teams_12 - 1
X_p1_12, time_p1_12 = optimal_formulation(N_teams_12, Win_, Wend_)
time_p1_12

33.7556068

In [21]:
N_teams_14 = 14
Win_ = 1
Wend_ = N_teams_14 - 1
X_p1_14, time_p1_14 = optimal_formulation(N_teams_14, Win_, Wend_)
time_p1_14

178.3809867

In [23]:
N_teams_16 = 16
Win_ = 1
Wend_ = N_teams_16 - 1
X_p1_16, time_p1_16 = optimal_formulation(N_teams_16, Win_, Wend_)
time_p1_16

352.7191117

In [25]:
N_teams_18 = 18
Win_ = 1
Wend_ = N_teams_18 - 1
X_p1_18, time_p1_18 = optimal_formulation(N_teams_18, Win_, Wend_)
time_p1_18

535.9281693

In [47]:
N_teams_20 = 20
Win_ = 1
Wend_ = N_teams_20 - 1
X_p1_20, time_p1_20 = optimal_formulation(N_teams_20, Win_, Wend_)
time_p1_20

3313.8499202

In [74]:
calendar = []
for w in 1:3
    push!(calendar, calendar_week(X_p1, w, 4))
end

In [75]:
obj_calendar = ComputeObjectiveValue(calendar, 3, 4)

0.9444444444444444

## Preliminary attempt 2: make it scale to 8 teams

Here I want to put our other approach, where we made it scale to 8 teams

## Approach 1: Optimize 4 weeks at a time

In [11]:
function Optimize4WeeksAndFreeze(Y_old, Win, Wend, solvetime)

    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j, 0 otherwise

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D) + sum(y[i, j] for i = 1:N, j = 1:N)) ### HAMZA: why do we add sum(y[i,j]) in the objective function

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d_])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d_] - 1)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == Wend - Win + 1)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = Win:Wend))

    # if team i played team j, team j played team i
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] == y[j, i])
    
    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in Win:Wend, d in 1:D) <= 1 - Y_old[i, j])

    # OPTIMIZE
    solvetime += @elapsed optimize!(model)

    # update old X, Y before returning them
    Y_old = Y_old .+ value.(y)

    return value.(x), Y_old, solvetime

end

Optimize4WeeksAndFreeze (generic function with 1 method)

In [12]:
Win = 1
Wend = 4

# first time, Y_old initialized to empty
Y_old_1 = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar_1 = []

# initialize solvetime = 0
time_1 = 0

println("Initial week: ", Win, ", end week: ", Wend)
println("#############################################")

Initial week: 1, end week: 4
#############################################


In [13]:
# get value of X and Y
X_1, Y_old_1, time_1 = Optimize4WeeksAndFreeze(Y_old_1, Win, Wend, time_1);

In [14]:
for w in Win:Wend
    push!(calendar_1, calendar_week(X_1, w, N))
end

In [15]:
Win_list = [1, 5, 9, 13, 17]
Wend_list = [4, 8, 12, 16, 19]

# first time, Y_old initialized to empty
Y_old_1 = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar_1 = []

# initialize solvetime = 0
time_1 = 0

for (Win, Wend) in zip(Win_list, Wend_list)

    println("Initial week: ", Win, ", end week: ", Wend)
    println("#############################################")

    # get value of X and Y
    X_1, Y_old_1, time_1 = Optimize4WeeksAndFreeze(Y_old_1, Win, Wend, time_1)

    for w in Win:Wend
        push!(calendar_1, calendar_week(X_1, w, N))
    end

end

Initial week: 1, end week: 4
#############################################
Initial week: 5, end week: 8
#############################################
Initial week: 9, end week: 12
#############################################
Initial week: 13, end week: 16
#############################################
Initial week: 17, end week: 19
#############################################


In [16]:
# total time elapsed
time_1

3083.990996977

In [17]:
# in this matrix all elements except the diagonal should be 1, in the diagonal they should all be 0
println(sum([Y_old_1[i,i] for i in 1:20]))

# all other elements should be 1 (400 elements in total, diagonal = 20 element is 0, sum should be 380)
println(sum(Y_old_1))

0.0
380.0


In [18]:
obj_calendar_1 = ComputeObjectiveValue(calendar_1, W, N)

56.95476190476191

In [19]:
for w in 1:19
    display(calendar_1[w])
end

Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,2,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,18.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,17.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,15.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,16.0
8,8,0.0,0.0,0.0,19.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,13.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,9.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,10.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,11.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,14.0
5,5,0.0,0.0,0.0,16.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,20.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,15.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,18.0,0.0
9,9,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,12.0
2,2,0.0,0.0,0.0,0.0,0.0,18.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,4,0.0,0.0,0.0,16.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,17.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,19.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,14.0
8,8,0.0,0.0,0.0,0.0,0.0,15.0,0.0
9,9,0.0,0.0,0.0,0.0,11.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4,4,0.0,0.0,0.0,0.0,11.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,3.0,0.0
6,6,0.0,0.0,0.0,0.0,7.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,6.0,0.0,0.0
8,8,0.0,0.0,0.0,14.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,10.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,18.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,20.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,4.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,3.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,6.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,9.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,12.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,7.0,0.0,0.0,0.0


...

Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,15.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,19.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,18.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,8.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,7.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,20.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,13.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,9.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,12.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,5.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,17.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,19.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,20.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,2.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,5.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,15.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,6.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,7.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,3.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,4.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,13.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,17.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,17.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,16.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,9.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,13.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,8.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,14.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,18.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,5.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,3.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,20.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,12.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,17.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,16.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,15.0,0.0


## Approach 2: Optimize 4 weeks at a time and freeze first, then move window

In [20]:
function Optimize4WeeksAndFreezeFirst(Y_old, Win, Wend, last_period, solvetime)

    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # dont print output
    set_silent(model)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D) + sum(y[i, j] for i = 1:N, j = 1:N)) ### HAMZA: zhy do we add sum(y[i,j]), does not impact the rest time of players

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d_])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d_] - 1)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == Wend - Win + 1)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = Win:Wend))

    @constraint(model, [i = 1:N, j = 1:N], y[i, j] == y[j, i])
    
    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in Win:Wend, d in 1:D) <= 1 - Y_old[i, j])

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    # OPTIMIZE
    solvetime += @elapsed optimize!(model)

    # update old Y before returning
    # differently than previous formulation, we update y only if (i, j) played in the first week of the interval
    if last_period
        # update all Y
        Y_old += value.(y)
    else   
        # update only the first week
        for i = 1:N
            for j = 1:N
                for d = 1:D
                    if value.(x[i, j, Win, d]) == 1
                        Y_old[i, j] += value.(y[i, j])
                    end
                end
            end
        end
    end

    return value.(x), Y_old, solvetime

end

Optimize4WeeksAndFreezeFirst (generic function with 1 method)

In [21]:
# define interval
interval = 3    # number of weeks in each interval in addition to first week

# first time, Y_old initialized to empty
Y_old_2 = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar_2 = []

# initialize solvetime to 0
time_2 = 0

for Win in 1:W-interval

    Wend = Win + interval

    println("Initial week: ", Win, ", end week: ", Wend)
    println("#############################################")

    # get value of X and Y
    X_2, Y_old_2, time_2 = Optimize4WeeksAndFreezeFirst(Y_old_2, Win, Wend, Wend == W, time_2)

    if Wend == W
        for w in Win:Wend
            push!(calendar_2, calendar_week(X_2, w, N)) 
        end
    else
        push!(calendar_2, calendar_week(X_2[:, :, Win:Win, :], Win, N))
    end

end

Initial week: 1, end week: 4
#############################################
Initial week: 2, end week: 5
#############################################
Initial week: 3, end week: 6
#############################################
Initial week: 4, end week: 7
#############################################
Initial week: 5, end week: 8
#############################################
Initial week: 6, end week: 9
#############################################
Initial week: 7, end week: 10
#############################################
Initial week: 8, end week: 11
#############################################
Initial week: 9, end week: 12
#############################################
Initial week: 10, end week: 13
#############################################
Initial week: 11, end week: 14
#############################################
Initial week: 12, end week: 15
#############################################
Initial week: 13, end week: 16
#############################################
Initial week: 

In [22]:
# total time elepsed
time_2

9881.218476704002

In [23]:
obj_calendar_2 = ComputeObjectiveValue(calendar_2, W, N)

51.678174603174604

In [24]:
for w in 1:W
    display(calendar_2[w])
end

Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,2,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,18.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,17.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,15.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,16.0
8,8,0.0,0.0,0.0,19.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,13.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,7.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,4.0,0.0
3,3,0.0,0.0,0.0,9.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,18.0,0.0
6,6,0.0,0.0,0.0,0.0,11.0,0.0,0.0
7,7,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,12.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,3.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,15.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,19.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,14.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,16.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,7.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,8.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,5.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,6.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,17.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,8.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,20.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,10.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,14.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,13.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,9.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,19.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,6.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,11.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,8.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,15.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,10.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,6.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,9.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,2.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,7.0,0.0,0.0,0.0


...

Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,14.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,11.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,12.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,13.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,16.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,19.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,20.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,9.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,8.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,5.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,7.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,2.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,18.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,3.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,16.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,12.0,0.0,0.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,5.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,15.0,0.0
3,3,0.0,0.0,0.0,0.0,13.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,6.0,0.0
5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,4.0,0.0
7,7,0.0,0.0,0.0,0.0,17.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,10.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,16.0,0.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,0.0,20.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,6.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,19.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,5.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,2.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,14.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,13.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,11.0


Row,Player,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,18.0
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,12.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,10.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,16.0
7,7,0.0,0.0,0.0,15.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,17.0
9,9,0.0,0.0,0.0,0.0,0.0,20.0,0.0


## Approach 3: Approach 1 + Random Restart
### Optimize 4 weeks at a time + random restart

In [27]:
# we build from what we previously did, taking as starting point the output of Approach 1
calendar_matrix_3 = GetCalendarMatrix(calendar_1, W, N)
obj_calendar_3 = obj_calendar_1
Y_old_3 = Y_old_1
time_3 = time_1

# how many times to repeat this random restart process
repeat = 10

# save obj_calendar_3
obj_3_iteration = []

# set seed
Random.seed!(1234)

for count in 1:repeat
    
    calendar_matrix_3, obj_calendar_3, Y_old_3, time_3 = random_restart(calendar_matrix_3, obj_calendar_3, Y_old_3, 3, time_3)

    append!(obj_3_iteration, obj_calendar_3)
    
end

Objective value of new calendar: 57.034126984126985
Objective value of new calendar: 57.034126984126985
Objective value of new calendar: 57.23253968253968
Objective value of new calendar: 56.20079365079364
Objective value of new calendar: 56.280158730158725
Objective value of new calendar: 56.20079365079364
Objective value of new calendar: 56.77936507936508
Objective value of new calendar: 56.85873015873016
Objective value of new calendar: 56.66031746031747
Objective value of new calendar: 56.77936507936508


In [28]:
x = range(1, repeat)
y = obj_3_iteration
plot(x, y, title = "Optimize 4 weeks and freeze all with random restart", xlabel = "Iteration", ylabel = "Objective value", label = "Objective value after each iteration")
savefig("greedy1_randrest.png")

"/Users/andreazanon/Optimization_Project/greedy1_randrest.png"

In [29]:
obj_calendar_3

56.20079365079364

In [30]:
time_3

5373.113900997001

In [31]:
# improvement over not having random restart
(obj_calendar_1 - obj_calendar_3) / obj_calendar_1

0.013238019592269229

In [32]:
for w in 0:18
    display(DataFrame(calendar_matrix_3[:, 7*w+1:7*w+7], ["Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monday"]))
end

Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,18.0,0.0
4,0.0,0.0,0.0,0.0,0.0,17.0,0.0
5,0.0,0.0,0.0,0.0,0.0,15.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,16.0
8,0.0,0.0,0.0,19.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,13.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,9.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,10.0,0.0
3,0.0,0.0,0.0,0.0,0.0,11.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,14.0
5,0.0,0.0,0.0,16.0,0.0,0.0,0.0
6,0.0,0.0,0.0,20.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,15.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,18.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,12.0
2,0.0,0.0,0.0,0.0,0.0,18.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,0.0,0.0,0.0,16.0,0.0,0.0,0.0
5,0.0,0.0,0.0,17.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,19.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,14.0
8,0.0,0.0,0.0,0.0,0.0,15.0,0.0
9,0.0,0.0,0.0,0.0,11.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,18.0,0.0,0.0,0.0
2,0.0,0.0,0.0,20.0,0.0,0.0,0.0
3,0.0,0.0,0.0,4.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,0.0,0.0,0.0
5,0.0,0.0,0.0,6.0,0.0,0.0,0.0
6,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,0.0,0.0,0.0,9.0,0.0,0.0,0.0
8,0.0,0.0,0.0,12.0,0.0,0.0,0.0
9,0.0,0.0,0.0,7.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,5.0
4,0.0,0.0,0.0,0.0,0.0,0.0,11.0
5,0.0,0.0,0.0,0.0,0.0,0.0,3.0
6,0.0,0.0,0.0,0.0,0.0,0.0,7.0
7,0.0,0.0,0.0,0.0,0.0,0.0,6.0
8,0.0,0.0,0.0,0.0,0.0,0.0,14.0
9,0.0,0.0,0.0,0.0,0.0,0.0,10.0


...

Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,10.0,0.0
2,0.0,0.0,0.0,0.0,0.0,17.0,0.0
3,0.0,0.0,0.0,0.0,0.0,12.0,0.0
4,0.0,0.0,0.0,0.0,0.0,5.0,0.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0
6,0.0,0.0,0.0,0.0,0.0,13.0,0.0
7,0.0,0.0,0.0,0.0,0.0,19.0,0.0
8,0.0,0.0,0.0,0.0,0.0,20.0,0.0
9,0.0,0.0,0.0,0.0,0.0,16.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,7.0
2,0.0,0.0,0.0,0.0,0.0,0.0,19.0
3,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,0.0,0.0,0.0,0.0,0.0,0.0,12.0
5,0.0,0.0,0.0,0.0,0.0,0.0,11.0
6,0.0,0.0,0.0,0.0,0.0,0.0,16.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,3.0
9,0.0,0.0,0.0,0.0,0.0,0.0,20.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,5.0,0.0,0.0,0.0
2,0.0,0.0,0.0,15.0,0.0,0.0,0.0
3,0.0,0.0,0.0,6.0,0.0,0.0,0.0
4,0.0,0.0,0.0,7.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0.0,0.0,0.0,3.0,0.0,0.0,0.0
7,0.0,0.0,0.0,4.0,0.0,0.0,0.0
8,0.0,0.0,0.0,13.0,0.0,0.0,0.0
9,0.0,0.0,0.0,17.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,17.0
2,0.0,0.0,0.0,0.0,0.0,0.0,16.0
3,0.0,0.0,0.0,0.0,0.0,0.0,9.0
4,0.0,0.0,0.0,0.0,0.0,0.0,13.0
5,0.0,0.0,0.0,0.0,0.0,0.0,8.0
6,0.0,0.0,0.0,0.0,0.0,0.0,14.0
7,0.0,0.0,0.0,0.0,0.0,0.0,18.0
8,0.0,0.0,0.0,0.0,0.0,0.0,5.0
9,0.0,0.0,0.0,0.0,0.0,0.0,3.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,0.0,0.0,0.0,20.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,12.0,0.0,0.0,0.0
6,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,0.0,0.0,0.0,17.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,16.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,15.0,0.0


## Approach 4: Approach 2 + random restart
### Optimize 4 weeks at a time, freeze first then move window + random restart

In [33]:
# we build from what we previously did, taking as starting point the output of Approach 2
calendar_matrix_4 = GetCalendarMatrix(calendar_2, W, N)
obj_calendar_4 = obj_calendar_2
Y_old_4 = Y_old_2
time_4 = time_2

# how many times to repeat this random restart process
repeat = 10

# save obj_calendar_4
obj_4_iteration = []

# set seed
Random.seed!(1234)

for count in 1:repeat
    
    calendar_matrix_4, obj_calendar_4, Y_old_4, time_4 = random_restart(calendar_matrix_4, obj_calendar_4, Y_old_4, 4, time_4)

    append!(obj_4_iteration, obj_calendar_4)
    
end

Objective value of new calendar: 52.828968253968256
Objective value of new calendar: 52.96388888888889
Objective value of new calendar: 54.11468253968253
Objective value of new calendar: 54.23214285714285
Objective value of new calendar: 54.31150793650793
Objective value of new calendar: 54.23214285714285
Objective value of new calendar: 55.51785714285714
Objective value of new calendar: 55.59722222222223
Objective value of new calendar: 55.32103174603174
Objective value of new calendar: 55.44007936507936


In [34]:
x = range(1, repeat)
y = obj_4_iteration
plot(x, y, title = "Optimize 4 weeks and freeze first with random restart", xlabel = "Iteration", ylabel = "Objective value", label = "Objective value after each iteration")
savefig("greedy2_randrest.png")

"/Users/andreazanon/Optimization_Project/greedy2_randrest.png"

In [35]:
time_4

11931.669387892001

In [36]:
obj_calendar_4

51.678174603174604

In [37]:
# improvement over not having random restart
(obj_calendar_2 - obj_calendar_4) / obj_calendar_2

0.0

In [38]:
for w in 0:18
    display(DataFrame(calendar_matrix_4[:, 7*w+1:7*w+7], ["Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monday"]))
end

Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,18.0,0.0
4,0.0,0.0,0.0,0.0,0.0,17.0,0.0
5,0.0,0.0,0.0,0.0,0.0,15.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,16.0
8,0.0,0.0,0.0,19.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,13.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,7.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0
3,0.0,0.0,0.0,9.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,0.0,0.0,0.0,18.0,0.0
6,0.0,0.0,0.0,0.0,11.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,12.0,0.0,0.0,0.0
9,0.0,0.0,0.0,3.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,15.0,0.0,0.0,0.0
2,0.0,0.0,0.0,19.0,0.0,0.0,0.0
3,0.0,0.0,0.0,14.0,0.0,0.0,0.0
4,0.0,0.0,0.0,16.0,0.0,0.0,0.0
5,0.0,0.0,0.0,7.0,0.0,0.0,0.0
6,0.0,0.0,0.0,8.0,0.0,0.0,0.0
7,0.0,0.0,0.0,5.0,0.0,0.0,0.0
8,0.0,0.0,0.0,6.0,0.0,0.0,0.0
9,0.0,0.0,0.0,17.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,11.0,0.0,0.0,0.0
2,0.0,0.0,0.0,8.0,0.0,0.0,0.0
3,0.0,0.0,0.0,15.0,0.0,0.0,0.0
4,0.0,0.0,0.0,10.0,0.0,0.0,0.0
5,0.0,0.0,0.0,6.0,0.0,0.0,0.0
6,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,0.0,0.0,0.0,9.0,0.0,0.0,0.0
8,0.0,0.0,0.0,2.0,0.0,0.0,0.0
9,0.0,0.0,0.0,7.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,8.0
2,0.0,0.0,0.0,0.0,0.0,0.0,20.0
3,0.0,0.0,0.0,0.0,0.0,0.0,10.0
4,0.0,0.0,0.0,0.0,0.0,0.0,14.0
5,0.0,0.0,0.0,0.0,0.0,0.0,13.0
6,0.0,0.0,0.0,0.0,0.0,0.0,9.0
7,0.0,0.0,0.0,0.0,0.0,0.0,19.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,6.0


...

Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,16.0
3,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,20.0
6,0.0,0.0,0.0,0.0,0.0,0.0,7.0
7,0.0,0.0,0.0,0.0,0.0,0.0,6.0
8,0.0,0.0,0.0,0.0,0.0,0.0,3.0
9,0.0,0.0,0.0,0.0,0.0,0.0,14.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,10.0
2,0.0,0.0,0.0,0.0,0.0,0.0,17.0
3,0.0,0.0,0.0,0.0,0.0,0.0,11.0
4,0.0,0.0,0.0,0.0,0.0,0.0,9.0
5,0.0,0.0,0.0,0.0,0.0,0.0,19.0
6,0.0,0.0,0.0,0.0,0.0,0.0,18.0
7,0.0,0.0,0.0,0.0,0.0,0.0,12.0
8,0.0,0.0,0.0,0.0,0.0,0.0,16.0
9,0.0,0.0,0.0,0.0,0.0,0.0,4.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,20.0,0.0,0.0,0.0
2,0.0,0.0,0.0,6.0,0.0,0.0,0.0
3,0.0,0.0,0.0,19.0,0.0,0.0,0.0
4,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,4.0,0.0,0.0,0.0
6,0.0,0.0,0.0,2.0,0.0,0.0,0.0
7,0.0,0.0,0.0,14.0,0.0,0.0,0.0
8,0.0,0.0,0.0,13.0,0.0,0.0,0.0
9,0.0,0.0,0.0,11.0,0.0,0.0,0.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,15.0
3,0.0,0.0,0.0,0.0,0.0,0.0,13.0
4,0.0,0.0,0.0,0.0,0.0,0.0,6.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,4.0
7,0.0,0.0,0.0,0.0,0.0,0.0,17.0
8,0.0,0.0,0.0,0.0,0.0,0.0,10.0
9,0.0,0.0,0.0,0.0,0.0,0.0,16.0


Row,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,18.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,12.0
5,0.0,0.0,0.0,0.0,0.0,0.0,10.0
6,0.0,0.0,0.0,0.0,0.0,0.0,16.0
7,0.0,0.0,0.0,15.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,17.0
9,0.0,0.0,0.0,0.0,0.0,20.0,0.0


## Approach 5: optimal formulation, set time limit

In [50]:
# use the same function defined in preliminary approach 1, but set time limit
N_teams = N
Win = 1
Wend = W
X_5, time_5 = optimal_formulation(N_teams, Win, Wend)

In [ ]:
calendar_5 = []

for w in Win:Wend
    push!(calendar_5, calendar_week(X_5, w))
end

In [ ]:
obj_calendar_5 = ComputeObjectiveValue(calendar_5, W, N)

In [ ]:
for w in 1:W
    display(calendar_5[w])
end

# Results

Plot results, meaning objective function and time needed to reach that objective function